In [3]:
import os
import requests
import json
from datetime import datetime
import requests
import pandas as pd
import urllib.request

In [4]:
def IG_LOGIN(username,password):
        try:
            with open('cookie.json', 'r') as openfile:
                # Reading from json file
                return json.load(openfile)
        except:
            link = 'https://www.instagram.com/accounts/login/'
            login_url = 'https://www.instagram.com/accounts/login/ajax/'

            time = int(datetime.now().timestamp())
            response = requests.get(link)
            csrf = response.cookies['csrftoken']

            payload = {
                'username': username,
                'enc_password': f'#PWD_INSTAGRAM_BROWSER:0:{time}:{password}',
                'queryParams': {},
                'optIntoOneTap': 'false'
            }



            login_header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36",

                "X-Requested-With": "XMLHttpRequest",
                "Referer": "https://www.instagram.com/accounts/login/",
                "x-csrftoken": csrf
            }

            login_response = requests.post(login_url, data=payload, headers=login_header)
            json_data = json.loads(login_response.text)


            if json_data["authenticated"]:
                cookies = login_response.cookies
                cookie_jar = cookies.get_dict()

                json_object = json.dumps(cookie_jar, indent = 4)
                with open("cookie.json", "w") as outfile:
                    outfile.write(json_object)
                return cookie_jar
            else:
                return "login failed ", login_response.text

In [5]:
import os
import requests
import json
from datetime import datetime
import requests

def go(base_url,user_id,headers,cookie_jar):
    res= requests.get(base_url.format(user_id),headers=headers, cookies=cookie_jar)
    user_info = res.json()
    for i in user_info["data"]["user"]["edge_owner_to_timeline_media"]["edges"]:
        if i["node"]["is_video"] == True:
            a.append(i["node"]["video_url"])
            b.append(i["node"]["display_url"])
    return str(user_info["data"]["user"]["edge_owner_to_timeline_media"]["page_info"]["end_cursor"]).replace("==","")

def reels(username,password,target,no_of_posts : int):
    global a
    a=[]
    global b
    b =[]
    cookie_jar = IG_LOGIN(username,password)
    headers = {
            'user-agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 12_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 105.0.0.11.118 (iPhone11,8; iOS 12_3_1; en_US; en-US; scale=2.00; 828x1792; 165586599)'
        }
    csrf_token = cookie_jar['csrftoken']
    session_id = cookie_jar['sessionid']
    user = {}
    user_id_req = requests.get(f"https://www.instagram.com/{target}/?__a=1",headers=headers, cookies=cookie_jar).json()
    user_id = user_id_req['graphql']['user']['id']
    if username:
        if no_of_posts>12:
            base_url = f'https://www.instagram.com/graphql/query/?query_hash=69cba40317214236af40e7efa697781d&variables=%7B%22id%22%3A%22{user_id}%22%2C%22first%22%3A12%7D'
        else:
            base_url = f'https://www.instagram.com/graphql/query/?query_hash=69cba40317214236af40e7efa697781d&variables=%7B%22id%22%3A%22{user_id}%22%2C%22first%22%3A{no_of_posts}%7D'
        #valid user-agent
        
        headers = {
            'user-agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 12_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 105.0.0.11.118 (iPhone11,8; iOS 12_3_1; en_US; en-US; scale=2.00; 828x1792; 165586599)'
        }
        try:
            res= requests.get(base_url.format(user_id),headers=headers, cookies=cookie_jar)
            user_info = res.json()
            for i in user_info["data"]["user"]["edge_owner_to_timeline_media"]["edges"]:
                if i["node"]["is_video"] == True:
                    a.append(i["node"]["video_url"])
                else:
                    b.append(i["node"]["display_url"])
                    
            if no_of_posts>12:
                after = str(user_info["data"]["user"]["edge_owner_to_timeline_media"]["page_info"]["end_cursor"]).replace("==","")
                while True:
                    no_of_posts -= 12
                    base_url = f"https://www.instagram.com/graphql/query/?query_hash=69cba40317214236af40e7efa697781d&variables=%7B%22id%22%3A%22{user_id}%22%2C%22first%22%3A12%2C%22after%22%3A%22{after}%3D%3D%22%7D"
                    if no_of_posts>=0:
                        after = go(base_url,user_id,headers,cookie_jar) 
                    else:
                        break
            if os.path.isdir(target) == False:
                os.mkdir(target)
            directories = ["Images","Videos"]
            for drt in directories:
                if os.path.isdir(f"{target}/{drt}")==False:
                    os.mkdir(f"{target}/{drt}")
            c1 = len(a)
            c2 = len(b)
            if c1>c2:
                c = c1
            else:
                c = c2
            
            for j in range(c):
                if c1!=0:
                    r = requests.get(a[j], allow_redirects=True)
                    print(f"Video_{j}")
                    open(f'./{target}/Videos/Video_{j}.mp4', 'wb').write(r.content)
                    c1 -=1
                if c2!=0:
                    r_img = requests.get(b[j], allow_redirects=True)
                    print(f"Image_{j}")
                    open(f'./{target}/Images/img_{j}.png', 'wb').write(r_img.content)
                    c2 -=1
        except Exception as e:
            print("getting user failed, due to '{}'".format(e.message))

In [9]:
reels(username = USERNAME,password = PASSWORD ,target ="motivationaldaru",no_of_posts=5)

Video_0
Video_1
Video_2
Video_3
Video_4
